In [1]:
import inseq
import pandas as pd
import warnings
import byt5_model
import torch

warnings.filterwarnings("ignore")

In [ ]:
!pwd
!pip3 install inseq

In [2]:
import platform 
platform.python_version()

'3.10.10'

In [3]:
class InseqAttributer:
    def __init__(self, model="./models/test/", attribution_method="input_x_gradient") -> None:
        """Create an object that loads a given model through Inseq with a given attribution method"""
        self.model = inseq.load_model(model, attribution_method)
    
    def attribute(self, inp:str, out:str=None):
        """Use the Inseq model to generate feature attributions using a given input or input & output"""
        if inp and out:
            inp_out = (inp, out)
        else:
            inp_out = (inp,)

        inseq_out = self.model.attribute(*inp_out,
                                         attribute_target=True,
                                         step_scores=["probability"]
                                        )
        inseq_out.show()


def predict_on_data(data, model_config_path, spaces=True, device='auto'):
    if device == 'auto':
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    elif device == 'gpu' or device == 'cuda':
        device = torch.device("cuda")
    elif device == 'cpu':
        device = torch.device("cpu")
    else:
        print('Set device to auto, cpu, or gpu/cuda.')
        return


    if spaces:
        data['inputs'] = data['lemma'] + ' ' + data['features']
    else:
        data['inputs'] = data['lemma'] + data['features']


    data_gen_comparison = byt5_model.comparer(data, byt5_model.generate(model_config_path, data, device))
    data = pd.concat([data, data_gen_comparison], axis=1).rename(columns={'labels': 'plural'})

    correct = []
    for idx, row in data.iterrows():
        if row['Expected'] == row['Predicted']:
            correct.append('correct')
        else:
            correct.append('incorrect')

    data = data.assign(correct=correct)

    return (data[data.correct == 'incorrect'], data[data.correct == 'correct'])

def construct_character_spans(elements):
    spans = []
    for idx, char in enumerate(elements):
        # The capital A with accents denote the start of a two-bytes wide character, for turkish
#         These are the characters: ÇĞİÖŞÜçğıöşü
        if char.token in 'ÃÄÅ':
            spans.append((idx, idx+2))
    
    return spans


def inseq_on_predictions(samples, inseq_model, expected=False):
    
    count = 0
    for idx, sample in samples.iterrows():
        count += 1
        if count > 1:
            print('\n')
        print('='*80)

        inp = sample['lemma'] + ' ' + sample['features']
        if expected:
            print('Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)')
            output_type = 'gold'
            out = sample['Expected']
        else:
            print('Testing Predicted (pred) outcome (what did the model take from the input during generation)')
            output_type = 'pred'
            out = sample['Predicted']

        print('Sample: {}\nIndex: {}\nThis prediction is {}\nInput -> output ({}): \'{}\' -> \'{}\''.format(
             count, idx, sample['correct'], output_type, inp, out
        ))
        if output_type == 'pred':
            print(f"The expected or gold output would be '{sample['Expected']}'")
            contrast = inseq_model.model.encode(sample['Expected'], as_targets=True)


        print('='*80)
        
        # Figure out where the plural suffix is in the word. We only need to attribute from there on out.
        plural_suffix_start_index = max([-1] + [out.find(plural_suffix) for plural_suffix in ['ler', 'lar'] if plural_suffix in out])

        # If aggregation on the target is disabled, the offset should be not the length of the string, but the byte-length.
        # For the attribution start and end, the offset should be compute as byte-length, instead of utf-8 string length
#         plural_suffix_start_index = len(out[:plural_suffix_start_index].encode('utf-8'))
#         print(f"{out=}, {plural_suffix_start_index=}")

        # These suffixes are always 3 characters long ('ler' or 'lar')
        plural_suffix_end_index = plural_suffix_start_index + 3

        if plural_suffix_start_index > 0:  # The suffix is not always present in the output
            out = inseq_model.model.attribute(
                inp,
                out,
                show_progress=False,
#                 attr_pos_start=plural_suffix_start_index,  # These are unfortunately disabled, as they seem to interfere with the aggregation
#                 attr_pos_end=plural_suffix_end_index,    # This causes conflicts with the aggregation.
                attribute_target=True,
                step_scores=["probability"],
            )
        else:
            out = inseq_model.model.attribute(
                inp,
                out,
                show_progress=False,
                attribute_target=True,
                step_scores=["probability"],
            )
        
        # Construct source and target spans to aggregate characters that are two bytes wide
        source_spans = construct_character_spans(out.sequence_attributions[0].source) or None
        target_spans = construct_character_spans(out.sequence_attributions[0].target) or None
        print(source_spans, target_spans)
        
        if source_spans or target_spans:
#         if source_spans:
            out=inseq.data.aggregator.ContiguousSpanAggregator.aggregate(out.sequence_attributions[0], 
                                                                    source_spans=source_spans, 
                                                                    target_spans=target_spans)
        out.show()
            

## Turkish

Focus on relation between last vowel of the stem and the first vowel of the suffix. We expect the last vowel to be salient in predicting the first vowel of the suffix. We focus on plural nouns.

In [4]:
model_tur_finetuned = "./drive-symlink/NLP_project_morphological_inflection/finetuned_tur_3"
inseq_tur_finetuned = InseqAttributer(model_tur_finetuned, "input_x_gradient")

In [ ]:
# sample_runner(data_sampler('./data/tur.gold', model_tur_finetuned, device='cpu'), inseq_tur_finetuned, expected=True)

In [5]:
header_names = ['lemma', 'labels', 'features']

turkish_train = pd.read_csv('./data/tur_large.train', sep='\t', names=header_names)

In [6]:
turkish_train.head()

,lemma,labels,features
0,masör,masörlerdim,N;ARGNO1S;PL;PST
1,kışkırtmak,kışkırtmış mıyım,V;SG;1;POS;PST;INTR;LGSPEC03
2,küçük,küçük değildiler,ADJ;DECL;PL;3;NEG;PST;LGSPEC01
3,Yahudi,Yahudimizin,N;GEN;SG;PSS1P
4,bitirttirmek,bitirttirecek olacak mıymışım,V;PROSP;SG;1;POS;FUT;INTR;LGSPEC03


In [7]:
filtered_turkish = turkish_train[
    turkish_train['features'].str.startswith('N;')  # Ensure it is a noun
  & turkish_train['features'].str.contains('PL;')]  # And plural
print(filtered_turkish.size)
filtered_turkish.head()

1803


,lemma,labels,features
0,masör,masörlerdim,N;ARGNO1S;PL;PST
28,pipet,pipetlerimiz,N;NOM;PL;PSS1P
36,göt,götlerinin,N;GEN;PL;PSS3S
57,bozayı,bozayılarınızdan,N;ABL;PL;PSS2P
82,hırsız,hırsızlarımızı,N;ACC;PL;PSS1P


In [8]:
incorrect_turkish, correct_turkish = predict_on_data(filtered_turkish.head(40), model_tur_finetuned)
print(f"Number of incorrect inflections: {len(incorrect_turkish)}, and correct: {len(correct_turkish)}")

               Expected           Predicted
0           masörlerdim          masördünüz
28         pipetlerimiz       pipetlerimizi
36           götlerinin             götünüz
57     bozayılarınızdan    bozayılarımızdan
82       hırsızlarımızı      hırsızlarımızı
92        durumlarından       durumlarından
96       şapkacılarında      şapkacılarında
98        nöronlarından       nöronlarından
100        karınlarımız        karınlarımız
103       vurgularından       vurgularından
106     notiluslarımızı     notiluslarımızı
109          incilermiş        incilermişiz
110       duraklarmışız         duraklarmış
127       bohçalarmışım         bohçalarmış
132   çığırtkanlarımızı   çığırtkanlarımızı
144         atomlarının       atomlarınızda
147  ayrımcılıklardınız        ayrımcılıkta
156  seri katillerimize  seri katillerimizi
195         kirpilermiş       kirpilermişiz
199         çöllerinden         çöllerinden
Number of incorrect inflections: 21, and correct: 19


In [9]:
incorrect_attributions = inseq_on_predictions(incorrect_turkish, inseq_tur_finetuned, expected=False)

Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 1
Index: 0
This prediction is incorrect
Input -> output (pred): 'masör N;ARGNO1S;PL;PST' -> 'masördünüz'
The expected or gold output would be 'masörlerdim'
[(3, 5)] [(3, 5), (7, 9), (10, 12)]


,m,a,s,Ã¶,r,d,Ã¼,n,Ã¼,z,</s>
m,0.119,0.075,0.027,0.034,0.029,0.019,0.024,0.009,0.011,0.007,0.008
a,0.155,0.129,0.105,0.076,0.026,0.036,0.042,0.027,0.01,0.011,0.03
s,0.066,0.064,0.051,0.035,0.03,0.041,0.009,0.031,0.013,0.024,0.008
Ã¶,0.127,0.129,0.055,0.113,0.038,0.018,0.056,0.023,0.034,0.043,0.024
r,0.026,0.053,0.046,0.09,0.089,0.028,0.05,0.036,0.029,0.018,0.011
,0.036,0.043,0.024,0.03,0.056,0.019,0.017,0.011,0.025,0.019,0.018
N,0.021,0.035,0.021,0.026,0.034,0.011,0.016,0.017,0.024,0.046,0.02
;,0.026,0.052,0.035,0.032,0.063,0.027,0.044,0.016,0.039,0.056,0.049
A,0.014,0.012,0.011,0.008,0.012,0.027,0.009,0.01,0.01,0.013,0.014
R,0.034,0.028,0.054,0.035,0.027,0.034,0.099,0.054,0.085,0.075,0.079




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 2
Index: 28
This prediction is incorrect
Input -> output (pred): 'pipet N;NOM;PL;PSS1P' -> 'pipetlerimizi'
The expected or gold output would be 'pipetlerimiz'
None None


,p,i,p,e,t,l,e,r,i,m,i,z,i,</s>
p,0.059,0.064,0.041,0.077,0.055,0.025,0.023,0.028,0.043,0.045,0.032,0.028,0.018,0.02
i,0.055,0.058,0.078,0.032,0.035,0.039,0.017,0.011,0.039,0.046,0.022,0.033,0.028,0.025
p,0.132,0.043,0.061,0.058,0.081,0.018,0.064,0.031,0.024,0.104,0.063,0.039,0.033,0.083
e,0.039,0.039,0.041,0.038,0.032,0.027,0.047,0.017,0.025,0.044,0.019,0.03,0.014,0.02
t,0.084,0.041,0.131,0.05,0.078,0.054,0.028,0.063,0.074,0.031,0.041,0.046,0.022,0.066
,0.023,0.041,0.031,0.047,0.039,0.058,0.032,0.019,0.032,0.026,0.064,0.06,0.046,0.024
N,0.012,0.039,0.021,0.044,0.032,0.034,0.022,0.018,0.011,0.018,0.014,0.017,0.016,0.013
;,0.086,0.032,0.036,0.048,0.024,0.041,0.032,0.046,0.027,0.03,0.049,0.036,0.016,0.029
N,0.022,0.04,0.021,0.011,0.015,0.014,0.014,0.013,0.019,0.02,0.03,0.022,0.009,0.018
O,0.073,0.125,0.082,0.041,0.022,0.036,0.036,0.041,0.05,0.087,0.029,0.052,0.014,0.036




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 3
Index: 36
This prediction is incorrect
Input -> output (pred): 'göt N;GEN;PL;PSS3S' -> 'götünüz'
The expected or gold output would be 'götlerinin'
[(1, 3)] [(1, 3), (4, 6), (7, 9)]


...

,Ã§,a,r,Ä±,n,Ä±,z,d,a,n,</s>
Ã§,0.085,0.048,0.043,0.035,0.065,0.094,0.05,0.049,0.083,0.022,0.026
a,0.106,0.062,0.035,0.023,0.035,0.011,0.028,0.024,0.041,0.029,0.01
r,0.049,0.038,0.031,0.023,0.112,0.055,0.025,0.03,0.015,0.021,0.032
,0.023,0.024,0.036,0.02,0.03,0.027,0.021,0.016,0.027,0.012,0.02
N,0.022,0.034,0.043,0.012,0.032,0.013,0.039,0.024,0.017,0.018,0.012
;,0.054,0.079,0.079,0.049,0.039,0.025,0.034,0.03,0.02,0.042,0.035
A,0.04,0.017,0.022,0.023,0.011,0.02,0.014,0.011,0.021,0.01,0.014
R,0.069,0.094,0.082,0.052,0.045,0.05,0.038,0.062,0.013,0.09,0.093
G,0.113,0.072,0.091,0.131,0.061,0.023,0.066,0.029,0.039,0.136,0.106
N,0.027,0.04,0.03,0.024,0.066,0.04,0.021,0.019,0.012,0.021,0.013




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 20
Index: 348
This prediction is incorrect
Input -> output (pred): 'palyaço N;ARGNO1P;PL;PST' -> 'palyaçolardı'
The expected or gold output would be 'palyaçolardık'
[(5, 7)] [(5, 7), (12, 14)]


,p,a,l,y,a,Ã§,o,l,a,r,d,Ä±,</s>
p,0.074,0.131,0.084,0.042,0.021,0.06,0.054,0.024,0.023,0.05,0.019,0.011,0.012
a,0.081,0.173,0.136,0.021,0.101,0.052,0.068,0.019,0.008,0.019,0.009,0.016,0.012
l,0.036,0.083,0.034,0.035,0.024,0.026,0.008,0.007,0.006,0.013,0.016,0.007,0.011
y,0.075,0.034,0.076,0.084,0.067,0.069,0.024,0.035,0.029,0.04,0.018,0.011,0.047
a,0.078,0.038,0.077,0.027,0.124,0.039,0.027,0.017,0.013,0.016,0.019,0.012,0.015
Ã§,0.031,0.041,0.06,0.038,0.06,0.189,0.047,0.027,0.029,0.085,0.033,0.025,0.05
o,0.031,0.021,0.022,0.032,0.022,0.045,0.027,0.016,0.023,0.022,0.024,0.017,0.029
,0.025,0.027,0.033,0.041,0.041,0.024,0.051,0.023,0.027,0.03,0.022,0.019,0.016
N,0.014,0.013,0.01,0.013,0.008,0.006,0.025,0.025,0.016,0.024,0.019,0.019,0.022
;,0.025,0.013,0.015,0.029,0.015,0.017,0.033,0.03,0.028,0.049,0.021,0.03,0.021




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 21
Index: 349
This prediction is incorrect
Input -> output (pred): 'korsan N;GEN;PL;PSS3S' -> 'korsanlarında'
The expected or gold output would be 'korsanlarının'
None [(9, 11)]


,k,o,r,s,a,n,l,a,r,Ä±,n,d,a,</s>
k,0.057,0.071,0.066,0.082,0.084,0.088,0.097,0.037,0.039,0.065,0.101,0.063,0.049,0.028
o,0.108,0.11,0.048,0.048,0.062,0.038,0.058,0.051,0.04,0.06,0.038,0.012,0.016,0.032
r,0.154,0.038,0.118,0.097,0.097,0.125,0.082,0.027,0.014,0.069,0.085,0.017,0.079,0.031
s,0.03,0.023,0.051,0.062,0.075,0.059,0.05,0.07,0.035,0.068,0.008,0.034,0.024,0.084
a,0.086,0.063,0.062,0.026,0.028,0.02,0.012,0.023,0.032,0.047,0.025,0.055,0.022,0.037
n,0.08,0.301,0.179,0.024,0.055,0.086,0.035,0.075,0.024,0.041,0.02,0.08,0.095,0.021
,0.022,0.044,0.031,0.041,0.021,0.051,0.018,0.027,0.029,0.012,0.012,0.014,0.039,0.026
N,0.021,0.02,0.027,0.025,0.015,0.023,0.032,0.021,0.028,0.016,0.018,0.01,0.011,0.019
;,0.029,0.023,0.033,0.067,0.053,0.049,0.035,0.028,0.034,0.063,0.078,0.029,0.017,0.028
G,0.018,0.009,0.014,0.015,0.011,0.013,0.01,0.014,0.016,0.014,0.013,0.015,0.017,0.017


In [10]:
correct_attributions = inseq_on_predictions(correct_turkish, inseq_tur_finetuned, expected=False)

Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 1
Index: 82
This prediction is correct
Input -> output (pred): 'hırsız N;ACC;PL;PSS1P' -> 'hırsızlarımızı'
The expected or gold output would be 'hırsızlarımızı'
[(1, 3), (5, 7)] [(1, 3), (5, 7), (11, 13), (14, 16), (17, 19)]


,h,Ä±,r,s,Ä±,z,l,a,r,Ä±,m,Ä±,z,Ä±,</s>
h,0.075,0.114,0.083,0.06,0.061,0.042,0.034,0.046,0.024,0.041,0.014,0.013,0.029,0.024,0.014
Ä±,0.102,0.2,0.074,0.06,0.038,0.059,0.024,0.028,0.019,0.038,0.025,0.024,0.017,0.032,0.023
r,0.049,0.033,0.182,0.114,0.175,0.07,0.017,0.052,0.015,0.055,0.045,0.02,0.049,0.045,0.038
s,0.039,0.041,0.04,0.054,0.036,0.042,0.018,0.026,0.015,0.043,0.033,0.013,0.024,0.047,0.017
Ä±,0.046,0.056,0.047,0.079,0.116,0.035,0.023,0.031,0.028,0.034,0.048,0.032,0.028,0.026,0.024
z,0.032,0.033,0.169,0.095,0.057,0.083,0.019,0.059,0.07,0.041,0.027,0.024,0.039,0.051,0.017
,0.038,0.02,0.028,0.032,0.02,0.045,0.025,0.044,0.049,0.043,0.024,0.019,0.046,0.036,0.016
N,0.037,0.012,0.017,0.016,0.008,0.02,0.029,0.025,0.02,0.009,0.012,0.015,0.01,0.012,0.028
;,0.059,0.055,0.02,0.049,0.018,0.047,0.043,0.039,0.03,0.058,0.046,0.042,0.025,0.05,0.036
A,0.02,0.011,0.01,0.007,0.014,0.007,0.032,0.008,0.021,0.007,0.014,0.014,0.023,0.013,0.01




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 2
Index: 92
This prediction is correct
Input -> output (pred): 'durum N;ABL;PL;PSS2S' -> 'durumlarından'
The expected or gold output would be 'durumlarından'
None [(8, 10)]


,d,u,r,u,m,l,a,r,Ä±,n,d,a,n,</s>
d,0.088,0.08,0.136,0.038,0.129,0.026,0.062,0.073,0.05,0.069,0.064,0.078,0.137,0.016
u,0.041,0.208,0.063,0.074,0.06,0.055,0.036,0.035,0.023,0.031,0.022,0.035,0.067,0.016
r,0.327,0.072,0.059,0.098,0.028,0.051,0.111,0.234,0.084,0.084,0.067,0.19,0.076,0.018
u,0.035,0.096,0.04,0.063,0.045,0.086,0.091,0.021,0.026,0.035,0.052,0.011,0.011,0.022
m,0.06,0.054,0.098,0.147,0.094,0.042,0.061,0.051,0.071,0.119,0.084,0.099,0.087,0.087
,0.015,0.068,0.059,0.045,0.029,0.04,0.015,0.028,0.02,0.02,0.016,0.015,0.028,0.025
N,0.011,0.022,0.027,0.012,0.034,0.03,0.009,0.014,0.015,0.009,0.009,0.007,0.01,0.013
;,0.021,0.028,0.042,0.014,0.056,0.032,0.013,0.034,0.024,0.024,0.02,0.022,0.028,0.031
A,0.018,0.013,0.021,0.009,0.007,0.008,0.009,0.012,0.014,0.008,0.006,0.007,0.016,0.021
B,0.024,0.038,0.078,0.033,0.024,0.015,0.038,0.029,0.052,0.011,0.017,0.033,0.047,0.038




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 3
Index: 96
This prediction is correct
Input -> output (pred): 'şapkacı N;LOC;PL;PSS2S' -> 'şapkacılarında'
The expected or gold output would be 'şapkacılarında'
[(0, 2), (7, 9)] [(0, 2), (7, 9), (12, 14)]


...

,p,a,l,i,k,a,r,y,a,l,a,r,m,Ä±,Å,</s>
p,0.053,0.158,0.105,0.074,0.074,0.029,0.082,0.011,0.062,0.022,0.026,0.009,0.009,0.007,0.019,0.013
a,0.078,0.121,0.06,0.052,0.031,0.034,0.039,0.044,0.046,0.035,0.048,0.044,0.027,0.02,0.008,0.045
l,0.062,0.037,0.094,0.052,0.027,0.01,0.021,0.009,0.016,0.008,0.017,0.014,0.016,0.008,0.008,0.018
i,0.016,0.026,0.054,0.126,0.028,0.046,0.06,0.046,0.026,0.03,0.019,0.017,0.014,0.007,0.01,0.022
k,0.018,0.024,0.032,0.029,0.026,0.035,0.016,0.019,0.028,0.02,0.027,0.015,0.014,0.017,0.012,0.025
a,0.032,0.013,0.01,0.022,0.04,0.06,0.02,0.04,0.042,0.03,0.022,0.023,0.031,0.042,0.01,0.041
r,0.094,0.064,0.077,0.017,0.173,0.102,0.091,0.07,0.079,0.042,0.036,0.09,0.016,0.012,0.025,0.009
y,0.019,0.027,0.02,0.066,0.093,0.15,0.145,0.171,0.107,0.086,0.061,0.032,0.103,0.066,0.017,0.047
a,0.081,0.035,0.029,0.047,0.029,0.023,0.013,0.018,0.038,0.016,0.014,0.031,0.011,0.015,0.014,0.02
,0.023,0.024,0.016,0.014,0.021,0.021,0.025,0.033,0.02,0.023,0.036,0.012,0.024,0.031,0.021,0.009




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 18
Index: 352
This prediction is correct
Input -> output (pred): 'müge N;ACC;PL;PSS1P' -> 'mügelerimizi'
The expected or gold output would be 'mügelerimizi'
[(1, 3)] [(1, 3)]


,m,Ã¼,g,e,l,e,r,i,m,i,z,i,</s>
m,0.058,0.048,0.058,0.035,0.065,0.036,0.04,0.046,0.039,0.059,0.016,0.03,0.027
Ã¼,0.168,0.083,0.128,0.072,0.034,0.03,0.052,0.038,0.055,0.077,0.049,0.055,0.05
g,0.075,0.069,0.051,0.049,0.029,0.029,0.041,0.021,0.024,0.053,0.082,0.053,0.033
e,0.097,0.094,0.145,0.035,0.062,0.057,0.021,0.084,0.031,0.028,0.039,0.017,0.05
,0.035,0.045,0.069,0.091,0.056,0.053,0.091,0.016,0.034,0.037,0.044,0.041,0.023
N,0.046,0.014,0.02,0.021,0.014,0.037,0.014,0.026,0.016,0.031,0.017,0.033,0.015
;,0.045,0.051,0.027,0.039,0.059,0.02,0.024,0.027,0.02,0.022,0.021,0.025,0.014
A,0.016,0.015,0.018,0.014,0.016,0.015,0.015,0.009,0.011,0.013,0.013,0.012,0.008
C,0.086,0.081,0.027,0.136,0.03,0.04,0.033,0.066,0.027,0.053,0.025,0.038,0.058
C,0.071,0.05,0.025,0.048,0.046,0.023,0.051,0.028,0.03,0.039,0.05,0.026,0.024




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 19
Index: 355
This prediction is correct
Input -> output (pred): 'yanardağ N;DAT;PL;PSS1P' -> 'yanardağlarımıza'
The expected or gold output would be 'yanardağlarımıza'
[(7, 9)] [(7, 9), (12, 14), (15, 17)]


,y,a,n,a,r,d,a,Ä,l,a,r,Ä±,m,Ä±,z,a,</s>
y,0.041,0.096,0.054,0.03,0.047,0.028,0.023,0.066,0.011,0.014,0.013,0.048,0.018,0.013,0.026,0.03,0.021
a,0.105,0.152,0.084,0.034,0.048,0.072,0.051,0.033,0.01,0.011,0.033,0.017,0.019,0.011,0.03,0.035,0.028
n,0.033,0.043,0.029,0.024,0.054,0.016,0.048,0.032,0.021,0.018,0.01,0.049,0.03,0.043,0.035,0.006,0.025
a,0.039,0.039,0.115,0.107,0.091,0.021,0.027,0.036,0.027,0.015,0.017,0.044,0.029,0.036,0.007,0.028,0.018
r,0.036,0.031,0.111,0.099,0.052,0.033,0.041,0.023,0.062,0.016,0.017,0.04,0.04,0.041,0.009,0.042,0.006
d,0.018,0.025,0.035,0.053,0.063,0.028,0.021,0.035,0.015,0.037,0.036,0.037,0.009,0.019,0.011,0.041,0.025
a,0.033,0.041,0.016,0.021,0.017,0.066,0.115,0.099,0.035,0.062,0.056,0.042,0.045,0.021,0.015,0.016,0.012
Ä,0.02,0.036,0.022,0.03,0.033,0.043,0.03,0.042,0.021,0.094,0.038,0.024,0.038,0.021,0.022,0.019,0.019
,0.021,0.01,0.021,0.018,0.015,0.037,0.027,0.024,0.021,0.033,0.014,0.023,0.014,0.039,0.026,0.028,0.026
N,0.028,0.008,0.006,0.014,0.01,0.025,0.024,0.015,0.009,0.012,0.012,0.014,0.032,0.025,0.015,0.014,0.014
